In [16]:
from  langgraph.graph import StateGraph,START,END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict
from dotenv import load_dotenv

In [17]:
load_dotenv()
import os

In [18]:
api_key = os.getenv("API_KEY")

In [19]:
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=api_key)


In [20]:
# Create a state
class LLMState(TypedDict):
    question:str
    answer:str

In [21]:
# LLm functioning
def llm_qa(state: LLMState)-> LLMState:
    # Extract the question from state
    question = state['question']
    # form a prompt
    prompt = f"Answer the following question {question}"

    # Ask that question
    answer = model.invoke(prompt).content
    # Update the answeer
    state['answer'] = answer

    return state

In [22]:
# Create Graph
graph = StateGraph(LLMState)

# Add Nodes
graph.add_node('llm_qa',llm_qa)

# Add edges
graph.add_edge(START,'llm_qa')
graph.add_edge('llm_qa',END)

workflow = graph.compile()

In [25]:
# Execute 
inital_state = {'question':'how far is moon from earth'}
final_state = workflow.invoke(inital_state)
print(final_state)

{'question': 'how far is moon from earth', 'answer': "The distance between the Earth and the Moon varies because the Moon's orbit is an ellipse, not a perfect circle.\n\n*   **Average distance:** 384,400 kilometers (238,900 miles)\n*   **Closest point (perigee):** Around 363,104 kilometers (225,623 miles)\n*   **Farthest point (apogee):** Around 405,696 kilometers (252,088 miles)"}


In [26]:
model.invoke('how far is moon from earth').content

"The distance between the Earth and the Moon isn't constant, as the Moon's orbit is elliptical.\n\n*   **Average distance:** 384,400 kilometers (238,900 miles)\n\n*   **Closest point (perigee):** Around 363,104 kilometers (225,623 miles)\n\n*   **Farthest point (apogee):** Around 405,696 kilometers (252,088 miles)"